# Retrieval Augmented Generation

Table of Contents
1. Idea
2. Naive Implementation
3. Graph-based Implementation

In [2]:
from language_models.proxy_client import BTPProxyClient
from language_models.models.embedding import SentenceTransformerEmbeddingModel
from language_models.settings import settings

In [ ]:
proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## Idea

## Naive Implementation

## Graph-based Implementation